In [7]:
# Creem la conex amb la db MYSQL
import mysql.connector

db = mysql.connector.connect(
    host="localhost",
    username="root",
    password="123321",
    database="tenfe"
)

In [8]:
import pandas as pd

In [9]:
# Llegim el excel i el guardem en un dataframe
df = pd.read_excel('../data/linies/R1.xlsx')

In [10]:
# Creem un cursor per poder fer les consultes
cursor = db.cursor()

In [11]:
# Extraiem el ID_PARADA del dataframe i el guardem en una llista
parades = df['ID_PARADA'].unique()

parades

# Inserim les parades a la taula PARADES
for parada in parades:
    # Comprovem que no existeixi la parada
    cursor.execute("SELECT * FROM stations WHERE name = %s", (parada,))
    result = cursor.fetchone()
    if result:
        continue
    # Inserim la parada
    sql = "INSERT INTO stations (name) VALUES (%s)"
    val = (parada,)
    cursor.execute(sql, val)


# Guardem els canvis
db.commit()

In [12]:
# Canviem del df cada nom de parada pel seu id
for _, fila in df.iterrows():
    sql = "SELECT id FROM stations WHERE name = %s"
    val = (fila['ID_PARADA'],)
    cursor.execute(sql, val)
    result = cursor.fetchone()
    # Canviem de la fila el nom de la parada pel seu id
    try:
        df.loc[df['ID_PARADA'] == fila['ID_PARADA'], 'ID_PARADA'] = result[0]
    except:
        print("error en la fila: ", fila)

df

,LINIA,ID_PARADA,HORA,ID_VIAJE,ORDEN
0,R1,2,05:48:00,1,1
1,R1,3,05:55:00,1,2
2,R1,4,06:00:00,1,3
3,R1,5,06:03:00,1,4
4,R1,6,06:07:00,1,5
...,...,...,...,...,...
3759,R1,17,22:53:00,190,5
3760,R1,18,22:51:00,190,4
3761,R1,19,22:47:00,190,3
3762,R1,20,22:43:00,190,2


In [13]:
# A tota la columna HORA la passem a string i posteriorment li treiem els ultims 3 caracters
df['HORA'] = df['HORA'].astype(str)
df['HORA'] = df['HORA'].str[:-3]

In [14]:
# Creem la linia a la taula linies si no existeix
for _, fila in df.iterrows():
    # Comprovem que no existeixi la linia
    cursor.execute("SELECT * FROM routes WHERE name = %s", (fila['LINIA'],))
    result = cursor.fetchone()
    if result:
        continue
    # Inserim la linia
    sql = "INSERT INTO routes (name) VALUES (%s)"
    val = (fila['LINIA'],)
    cursor.execute(sql, val)

In [15]:
# Inserim les dades de l'excel a la db
for _, row in df.iterrows():
    # Abans que res comprovem que no existeixi el registre
    cursor.execute("SELECT * FROM schedules WHERE station_id = %s AND time = %s AND train_num = %s AND stop_number = %s", (row['ID_PARADA'], row['HORA'], row['ID_VIAJE'], row['ORDEN']))
    result = cursor.fetchone()
    if result:
        continue
    # Inserim el registre
    sql = "INSERT INTO schedules (route_id, station_id, time, train_num, stop_number) VALUES (%s, %s, %s, %s, %s)"
    val = (row['LINIA'], row['ID_PARADA'], row['HORA'], row['ID_VIAJE'], row['ORDEN'])
    cursor.execute(sql, val)
    
db.commit()